In [64]:
import os
import numpy as np
import re
import requests
import pandas as pd
import html5lib
import time
from selenium import webdriver
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from urllib.request import urlopen
import warnings
warnings.filterwarnings("ignore")

import zipfile
import os
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

## Data Background

These datasets were scraped from [basketball reference](basketball-reference.com). 

The ```pergamestats.csv``` dataset contains the tables from Basketball Reference's NBA season per game player stat pages (e.g. [1979-80 NBA Player Stats: Per Game](https://www.basketball-reference.com/leagues/NBA_1980_per_game.html)), taking the 1979-80  to the 2023-24 seasons and putting them all in one table. Each of these tables has the per game stats of all players that season for each team they played for (as well as the combined stats for players who played for multiple teams). The combined dataset is also given a "Player Link" column, containing the subdirectory and filename from each player's Basketball Reference Player Page, which is used to help uniquely identify players (as each player has a unique filename).

The ```advancestats.csv``` dataset, similar to players1.csv, contains the tables from Basketball Reference's NBA season player advanced stat pages (e.g. [1979-80 NBA Player Stats: Advanced](https://www.basketball-reference.com/leagues/NBA_1980_advanced.html)) from 1979-80 to 2023-24.  

The ```allsalaries.csv``` dataset contains the "Salaries" table from a player's Basketball Reference page (e.g. [Kevin Garnett Salaries](https://www.basketball-reference.com/players/g/garneke01.html#all_all_salaries)), for each unique player in the ```pergamestats``` dataset. Each "Salaries" table contains the amount the player earned from their player contract, the team they last played for, the league of the team (which is always the NBA in this case) and the season for each season they played. This dataset was also contains the same "Player Link" column as ```pergamestats.csv```

In [65]:
PG_stats = pd.read_csv('pergamestats.csv')
salaries = pd.read_csv('allsalaries.csv')
adv_stats = pd.read_csv('advancedstats.csv')

In [66]:
# removing 1 row which just contains the name of the column for each value. 
# Seems to stem from Magic Johnson's salary table https://www.basketball-reference.com/players/j/johnsma02.html#all_all_salaries
salaries = salaries.drop(8856).reset_index()

In [67]:
# removing columns made up of nan values or those originally used for indexing 
PG_stats = PG_stats.drop(["Unnamed: 0", "Rk"], axis = 1)
adv_stats = adv_stats.drop(["Unnamed: 0", "Rk", "Unnamed: 19", "Unnamed: 24"], axis = 1)
salaries = salaries.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)

In [68]:
PG_stats.head(5)

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Player Link,Season
0,George Gervin,27.0,SAS,SG,78.0,NaN,37.6,13.1,24.9,0.528,...,5.2,2.6,1.4,1.0,3.3,2.7,33.1,"MVP-3,AS,NBA1",/players/g/gervige01.html,1979-80
1,World B. Free,26.0,SDC,SG,68.0,NaN,38.0,10.8,22.9,0.474,...,3.5,4.2,1.2,0.5,3.4,2.9,30.2,AS,/players/f/freewo01.html,1979-80
2,Adrian Dantley,24.0,UTA,SF,68.0,NaN,39.3,10.7,18.6,0.576,...,7.6,2.8,1.4,0.2,3.4,3.1,28.0,AS,/players/d/dantlad01.html,1979-80
3,Julius Erving,29.0,PHI,SF,78.0,78.0,36.1,10.7,20.7,0.519,...,7.4,4.6,2.2,1.8,3.6,2.7,26.9,"MVP-2,AS,NBA1",/players/e/ervinju01.html,1979-80
4,Moses Malone,24.0,HOU,C,82.0,NaN,38.3,9.5,18.9,0.502,...,14.5,1.8,1.0,1.3,3.7,2.6,25.8,"MVP-9,AS,NBA2",/players/m/malonmo01.html,1979-80


In [69]:
adv_stats.head(5)

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Player Link,Season
0,Kareem Abdul-Jabbar*,C,32,LAL,82,3143,25.3,0.639,0.001,0.344,...,9.5,5.3,14.8,0.227,4.8,2.4,7.2,7.3,/players/a/abdulka01.html,1979-80
1,Tom Abernethy,PF,25,GSW,67,1222,11.0,0.511,0.003,0.258,...,1.2,0.8,2.0,0.080,-1.0,-0.2,-1.2,0.2,/players/a/abernto01.html,1979-80
2,Alvan Adams,C,25,PHO,75,2168,19.2,0.571,0.002,0.270,...,3.1,3.9,7.0,0.155,1.7,1.9,3.6,3.1,/players/a/adamsal01.html,1979-80
3,Tiny Archibald*,PG,31,BOS,80,2864,15.3,0.574,0.023,0.548,...,5.9,2.9,8.9,0.148,1.4,-0.3,1.1,2.3,/players/a/architi01.html,1979-80
4,Dennis Awtrey,C,31,CHI,26,560,7.4,0.524,0.000,0.833,...,0.1,0.5,0.6,0.053,-2.3,0.9,-1.4,0.1,/players/a/awtrede01.html,1979-80


In [70]:
salaries.head(5)

,index,Season,Team,Lg,Salary,Player Link
0,0,1990-91,Portland Trail Blazers,NBA,"$395,000",/players/a/abdelal01.html
1,1,1991-92,Portland Trail Blazers,NBA,"$494,000",/players/a/abdelal01.html
2,2,1992-93,Boston Celtics,NBA,"$500,000",/players/a/abdelal01.html
3,3,1993-94,Boston Celtics,NBA,"$805,000",/players/a/abdelal01.html
4,4,1994-95,Sacramento Kings,NBA,"$650,000",/players/a/abdelal01.html


## Initial Data Manipulation

Let's combine the three datasets. ```PG_stats``` and ```adv_stats``` contain the predictive values while ```salaries``` contains the labels. First let's combine ```PG_stats``` and ```adv_stats```, then combine that with ```salaries```. We can uniquely identify each row by using the 'Player Link', 'Team', and 'Season' values.

In [71]:
left = PG_stats
# rows that indicate a player played for multiple teams are labeled differently in per game and advanced tables, changing it so they're the same
new_team = left["Team"].str.replace("2TM", "TOT").str.replace("3TM", "TOT").str.replace("4TM", "TOT").str.replace("5TM", "TOT")
left["Team"] = new_team

In [72]:
right = adv_stats
right["Team"] = right["Tm"]
right = right.drop("Tm", axis = 1)
for i in PG_stats.columns:
    if i in right.columns and i not in ["Team", "Player Link", "Season"]:
        right = right.drop(i, axis = 1)

In [73]:
all_vals = pd.merge(left, right, on = ["Team", "Player Link", "Season"])

We now have the ```all_vals``` table which contains all player stats. We also want just one row of data for each player for each season. Currently, for players who played for multiple teams in a season, there is a row of data for that player for each team they played for that season, as well as a row for their cumulative stats that season. Let's drop the individual team stats for these players, keeping just the cumulative stat row.

In [74]:
no_dupe_vals = all_vals.copy()

In [75]:
multi_teamers = all_vals["Team"] == "TOT"

In [76]:
mt_ids = all_vals[multi_teamers][["Player Link", "Season"]]

In [77]:
mt_matches = (all_vals["Player Link"] + all_vals["Season"]).isin(mt_ids["Player Link"] + mt_ids["Season"])

In [78]:
no_dupe_vals = no_dupe_vals.drop(all_vals[mt_matches][all_vals[mt_matches]["Team"] != "TOT"].index)

We now have a dataframe ```no_dupe_vals``` which has one row of data for each player for each season they played. However, it is still not the same size as ```salaries```. 

Let's manipulate the ```salaries``` next. First, we should address the non-numerical values under the "Salary" column. More specifically, some rows have either "< $Minimum" or "(TW)" as their values under the "Salary" column. Let's replace any instances of these values with the Two-Way Player salary for that year.

In [84]:
# Dropping a row where the salary value is NaN. 
# Stems from Tornike Shengelia's salary table https://www.basketball-reference.com/players/s/shengto01.html#all_all_salaries
# This is the only table that contains a null value out of any player I looked at
salaries = salaries.drop(15305)

In [85]:
no_null_slry = salaries[~salaries["Team"].isnull()]

In [86]:
# Manually inputting a table of Two-Way Salaries for each year
two_way_sals = pd.DataFrame({"Season": ["2017-18", "2018-19", "2019-20", "2020-21", "2021-22", "2022-23", "2023-24"],
                            "TW Salary": [815615 // 2, 838464 // 2, 898310 // 2, 898310 // 2, 925258 // 2, 995928 // 2, 1119563 // 2]})

In [87]:
non_num_sals = no_null_slry[np.logical_or(np.logical_or(no_null_slry["Salary"] == "< $Minimum", no_null_slry["Salary"] == "(TW)"), no_null_slry["Salary"].isnull())]

In [88]:
slry_TW_added = no_null_slry.copy()
for i in non_num_sals.index:
    season = non_num_sals["Season"][i]
    TW_salary = two_way_sals["TW Salary"][two_way_sals["Season"] == season].values[0]  
    new_sal_col = slry_TW_added["Salary"]
    new_sal_col[i] = str(TW_salary)
    slry_TW_added["Salary"] = new_sal_col

Now that we have ```slry_TW_added```, with all numerical strings values in its "Salary" column, let's turn these strings into integers by removing their non-numbers, then changing the dtype into int.

In [89]:
slry_TW_added["Salary"] = slry_TW_added["Salary"].str.replace("$", "").str.replace(",", "").str.replace("\(TW\)", "").astype(int)

Now, we want each season and player combination to have its own unique row in our salaries table. There are some season and player combinations which have multiple rows, as these players were payed by multiple teams in that season. Let's combine these rows so we get a player's total salary for each season. Note that for these new combined rows, we will be listing the team as the last team for that season in the original table. 

In [90]:
dtf = pd.DataFrame({"identifiers": no_null_slry["Player Link"] + no_null_slry["Season"], "ones": np.ones(len(no_null_slry))})
salaries_multi_rowers = dtf.groupby("identifiers").sum()[dtf.groupby("identifiers").sum()["ones"] > 1].index

In [91]:
no_dupe_slry = slry_TW_added.drop("index", axis = 1).copy()
for i in salaries_multi_rowers:
    player_link = i[:len(i) - 7]
    season = i[len(i) - 7:]
    match_rows = (no_dupe_slry["Season"] == season) * (no_dupe_slry["Player Link"] == player_link)
    match_db = no_dupe_slry.copy()[match_rows]
    no_dupe_slry = no_dupe_slry.drop(match_db.index)
    no_dupe_slry.loc[no_dupe_slry.index[-1] + 1] = [season, match_db["Team"][match_db.index[-1]], "NBA", sum(match_db["Salary"]), player_link]

In [24]:
none_links = salaries[salaries["Salary"].isnull()]["Player Link"]

In [25]:
no_sal_players = no_dupe_vals[no_dupe_vals["Player Link"].isin(none_links)]
maybe_sal_players = no_dupe_vals[~no_dupe_vals["Player Link"].isin(none_links)]


in_slry = (maybe_sal_players["Player Link"] + maybe_sal_players["Season"]).isin(no_dupe_slry["Player Link"] + no_dupe_slry["Season"])
no_sal_season = maybe_sal_players[~in_slry]
sal_seasons = maybe_sal_players[in_slry]

In [26]:
in_all_vals = (no_dupe_slry["Player Link"] + no_dupe_slry["Season"]).isin(no_dupe_vals["Player Link"] + no_dupe_vals["Season"])
yes_stat_slry = no_dupe_slry[in_all_vals]
no_stat_slry = no_dupe_slry[~in_all_vals]

Some player and season combinations in the salaries table seem to not have a corresponding stats row in ```all_stats```. For now, let's make a new table ```all_slry``` that adds these player and season combinations, with all values being null besides the "Player Link" and "Season" values.

In [27]:
all_slry_vals = sal_seasons.copy()
for i in no_stat_slry.index:
    all_slry_vals.loc[all_slry_vals.index[-1] + 1] = [None, None, None, None, None, None, None, None, None, None,
                                           None, None, None, None, None, None, None, None, None, None,
                                           None, None, None, None, None, None, None, None, None, None,
                                           no_stat_slry["Player Link"][i], no_stat_slry["Season"][i], None, 
                                            None, None, None, None, None, None, None,None, None, None, None, 
                                            None, None, None, None, None, None, None, None]

In [28]:
vals_sals_comb = pd.merge(all_slry_vals.drop("Team", axis = 1), no_dupe_slry, on = ["Player Link", "Season"])

In [29]:
vals_sals_comb

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Team,Lg,Salary
0,Larry Bird,23.0,PF,82.0,82.0,36.0,8.5,17.8,0.474,0.7,...,5.6,11.2,0.182,3.0,1.5,4.5,4.8,Boston Celtics,NBA,650000
1,Magic Johnson,20.0,SG,77.0,NaN,36.3,6.5,12.3,0.530,0.1,...,4.3,10.5,0.180,3.1,1.7,4.8,4.8,Los Angeles Lakers,NBA,460000
2,Magic Johnson,21.0,SG,37.0,NaN,37.1,8.4,15.9,0.532,0.1,...,2.6,6.4,0.225,5.6,3.3,8.8,3.8,Los Angeles Lakers,NBA,460000
3,Larry Bird,24.0,PF,82.0,82.0,39.5,8.8,18.3,0.478,0.2,...,6.1,10.8,0.160,2.5,1.8,4.3,5.2,Boston Celtics,NBA,650000
4,Larry Bird,25.0,PF,77.0,58.0,38.0,9.2,18.4,0.503,0.1,...,5.7,12.5,0.205,4.4,2.3,6.6,6.4,Boston Celtics,NBA,650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17480,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Toronto Raptors,NBA,4208107
17481,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minnesota Timberwolves,NBA,2376667
17482,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New York Knicks,NBA,3421709
17483,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Bulls,NBA,1362611


Next, let's make the "Salary" column consist entirely of integers. Right now, the "Salary" column consists of Strings with dollar amounts (e.g. "$1,000,000") or containing either "(TW)" or "<minimum". "(TW)" means they were on a two-way contract and "<minimum" means they recieved less than the minimum salary, which i'm assuming means less than a two-way contract. I will be replacing instances of either of these values with the two-way contract salary.

I will be manually inputing a table of two-way salaries ```TW_sals``` to aid me.

After filtering our ```no_dupes``` table to player and season combinations that are in ```salaries```, the resulting table ```all_stats``` has in fact less rows than ```salaries```. This means there are player and season combinations that we have salary data for but no stat data for. After looking at a couple examples, I can assume that these salary data points result from either the player playing no minutes that season, or when the player gets waived from the team but they have a guaranteed contract that the team must continue paying. For now, we can fix this by adding rows to ```all_stats``` which have player and season combinations that appear in ```salaries``` that were not in the table originally, and having all the stat values for these rows be nulls.

## Barrier

We will now take this ```vals_sals_comb``` table, and further manipulating to make it suitable for training.

When it comes to player and season combinations who have a salary but no stats, we can either try filling in the stat values with appropriate values (such as all zeroes), or just drop them. I will be dropping them as I am more interested in trying to predict in a season they play than what they might earn in the seasons right after they stop playing.

In [32]:
val_sal_w_vals = vals_sals_comb[~vals_sals_comb["Age"].isnull()]

We no longer have any rows with all null stat values, but some stat columns do still have null values. Let's look identify these columns.

In [33]:
np.sum(val_sal_w_vals.isnull(), axis = 0)

Player             0
Age                0
Pos                0
G                  0
GS                 2
MP                 0
FG                 0
FGA                0
FG%               48
3P                 0
3PA                0
3P%             2048
2P                 0
2PA                0
2P%               91
eFG%              48
FT                 0
FTA                0
FT%              466
ORB                0
DRB                0
TRB                0
AST                0
STL                0
BLK                0
TOV                0
PF                 0
PTS                0
Awards         14327
Player Link        0
Season             0
PER                3
TS%               43
3PAr              48
FTr               48
ORB%               3
DRB%               3
TRB%               3
AST%               3
STL%               3
BLK%               3
TOV%              38
USG%               3
OWS                0
DWS                0
WS                 0
WS/48              3
OBPM         

A few columns have null values. However, the null values in the "Awards" column can actually be thought of as a value, as that just means the player did not recieve any awards that year, so these null values are not a problem. 

Most of the other null values seem to be in percentage columns, and I can assume that a null percentage means there was no attempts of that shot type. Let's replace these null percentages that arrise from having no attempts with zeroes.

In [36]:
nonull_val_sal = val_sal_w_vals.copy()

In [37]:
no_ft_attempts = nonull_val_sal["FTA"] == 0
# free throw percentage
modified_FTP = nonull_val_sal["FT%"]
modified_FTP[no_ft_attempts] = 0
nonull_val_sal["FT%"] = modified_FTP

In [38]:
no_fg_attempts = nonull_val_sal["FGA"] == 0
# field goal percentage
modified_FGP = nonull_val_sal["FG%"]
modified_FGP[no_fg_attempts] = 0
nonull_val_sal["FG%"] = modified_FGP
# effective field goal percentage
modified_EFG = nonull_val_sal["eFG%"]
modified_EFG[no_fg_attempts] = 0
nonull_val_sal["eFG%"] = modified_EFG

In [39]:
no_2pt_attempts = nonull_val_sal["2PA"] == 0
# 2 pointer percentage
modified_2PP = nonull_val_sal["2P%"]
modified_2PP[no_2pt_attempts] = 0
nonull_val_sal["2P%"] = modified_2PP

In [40]:
no_3pt_attempts = nonull_val_sal["3PA"] == 0
# 3 pointer percentage
modified_3PP = nonull_val_sal["3P%"]
modified_3PP[no_3pt_attempts] = 0
nonull_val_sal["3P%"] = modified_3PP

In [41]:
no_fg_or_ft_attempts = (nonull_val_sal["FGA"] == 0) * (nonull_val_sal["FTA"] == 0)
# True shooting
modified_TS = nonull_val_sal["TS%"]
modified_TS[no_fg_or_ft_attempts] = 0
nonull_val_sal["TS%"] = modified_TS

In [42]:
no_fg_attempts = nonull_val_sal["FGA"] == 0
# 3 Pointer rate
modified_3PAr = nonull_val_sal["3PAr"]
modified_3PAr[no_fg_attempts] = 0
nonull_val_sal["3PAr"] = modified_3PAr
# Free throw rate
modified_FTr = nonull_val_sal["FTr"]
modified_FTr[no_fg_attempts] = 0
nonull_val_sal["FTr"] = modified_FTr

In [43]:
zero_TOp_denom = (nonull_val_sal["FGA"] + nonull_val_sal["TOV"] + .44 * nonull_val_sal["FTA"]) == 0
# turnover percentage
modified_TOVp = nonull_val_sal["TOV%"]
modified_TOVp[zero_TOp_denom] = 0
nonull_val_sal["TOV%"] = modified_TOVp

In [44]:
zero_min = nonull_val_sal["MP"] == 0
# PER
modified_PER = nonull_val_sal["PER"]
modified_PER[zero_min] = 0
nonull_val_sal["PER"] = modified_PER
# Offensive Rebound Percentage
modified_ORBp = nonull_val_sal["ORB%"]
modified_ORBp[zero_min] = 0
nonull_val_sal["ORB%"] = modified_ORBp
# Defensive Rebound Percentage
modified_DRBp = nonull_val_sal["DRB%"]
modified_DRBp[zero_min] = 0
nonull_val_sal["DRB%"] = modified_DRBp
# Total Rebound Percentage
modified_TRBp = nonull_val_sal["TRB%"]
modified_TRBp[zero_min] = 0
nonull_val_sal["TRB%"] = modified_TRBp
# Assist Percentage
modified_ASTp = nonull_val_sal["AST%"]
modified_ASTp[zero_min] = 0
nonull_val_sal["AST%"] = modified_ASTp
# Steal Percentage
modified_STLp = nonull_val_sal["STL%"]
modified_STLp[zero_min] = 0
nonull_val_sal["STL%"] = modified_STLp
# Block Percentage
modified_BLKp = nonull_val_sal["BLK%"]
modified_BLKp[zero_min] = 0
nonull_val_sal["BLK%"] = modified_BLKp
# Usage Percentage
modified_USGp = nonull_val_sal["USG%"]
modified_USGp[zero_min] = 0
nonull_val_sal["USG%"] = modified_USGp
# Win shares per 48 minutes
modified_WS48 = nonull_val_sal["WS/48"]
modified_WS48[zero_min] = 0
nonull_val_sal["WS/48"] = modified_WS48

In [45]:
np.sum(nonull_val_sal.isnull(), axis = 0)

Player             0
Age                0
Pos                0
G                  0
GS                 2
MP                 0
FG                 0
FGA                0
FG%                0
3P                 0
3PA                0
3P%                0
2P                 0
2PA                0
2P%                0
eFG%               0
FT                 0
FTA                0
FT%                0
ORB                0
DRB                0
TRB                0
AST                0
STL                0
BLK                0
TOV                0
PF                 0
PTS                0
Awards         14327
Player Link        0
Season             0
PER                0
TS%                0
3PAr               0
FTr                0
ORB%               0
DRB%               0
TRB%               0
AST%               0
STL%               0
BLK%               0
TOV%               0
USG%               0
OWS                0
DWS                0
WS                 0
WS/48              0
OBPM         

We've succeeded in removing all percentage-related null values. That leaves us with 2 null values in the "GS" columns.

In [46]:
nonull_val_sal[nonull_val_sal["GS"].isnull()]

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Team,Lg,Salary
1,Magic Johnson,20.0,SG,77.0,NaN,36.3,6.5,12.3,0.530,0.1,...,4.3,10.5,0.180,3.1,1.7,4.8,4.8,Los Angeles Lakers,NBA,460000
2,Magic Johnson,21.0,SG,37.0,NaN,37.1,8.4,15.9,0.532,0.1,...,2.6,6.4,0.225,5.6,3.3,8.8,3.8,Los Angeles Lakers,NBA,460000


I am not exactly sure why these two rows specifically have null "GS" values. However, I am fairly certain Magic Johnson started in all the games he played in his first two seasons, so let's just replace these values with the amount of games he played those seasons.

In [51]:
modified_GS = nonull_val_sal["GS"]
modified_GS[1] = 77
modified_GS[2] = 37
nonull_val_sal["GS"] = modified_GS

In [52]:
nonull_val_sal

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Team,Lg,Salary
0,Larry Bird,23.0,PF,82.0,82.0,36.0,8.5,17.8,0.474,0.7,...,5.6,11.2,0.182,3.0,1.5,4.5,4.8,Boston Celtics,NBA,650000
1,Magic Johnson,20.0,SG,77.0,77.0,36.3,6.5,12.3,0.530,0.1,...,4.3,10.5,0.180,3.1,1.7,4.8,4.8,Los Angeles Lakers,NBA,460000
2,Magic Johnson,21.0,SG,37.0,37.0,37.1,8.4,15.9,0.532,0.1,...,2.6,6.4,0.225,5.6,3.3,8.8,3.8,Los Angeles Lakers,NBA,460000
3,Larry Bird,24.0,PF,82.0,82.0,39.5,8.8,18.3,0.478,0.2,...,6.1,10.8,0.160,2.5,1.8,4.3,5.2,Boston Celtics,NBA,650000
4,Larry Bird,25.0,PF,77.0,58.0,38.0,9.2,18.4,0.503,0.1,...,5.7,12.5,0.205,4.4,2.3,6.6,6.4,Boston Celtics,NBA,650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16378,Danny Green,36.0,SG,2.0,0.0,9.0,0.0,1.0,0.000,0.0,...,0.0,0.0,-0.035,-7.4,0.8,-6.6,0.0,Philadelphia 76ers,NBA,200000
16379,Ron Harper Jr.,23.0,PF,1.0,0.0,4.0,0.0,0.0,0.000,0.0,...,0.0,0.0,0.087,-11.7,2.8,-8.8,0.0,Toronto Raptors,NBA,559781
16380,Justin Jackson,28.0,SF,2.0,0.0,0.5,0.0,0.0,0.000,0.0,...,0.0,0.0,0.031,-6.3,-1.2,-7.5,0.0,Minnesota Timberwolves,NBA,116075
16381,Dmytro Skapintsev,25.0,C,2.0,0.0,1.0,0.0,0.5,0.000,0.0,...,0.0,0.0,-0.483,-16.0,-9.8,-25.9,0.0,New York Knicks,NBA,559781


## Feature Engineering

In [53]:
nonull_val_sal.dtypes

Player          object
Age            float64
Pos             object
G              float64
GS             float64
MP             float64
FG             float64
FGA            float64
FG%            float64
3P             float64
3PA            float64
3P%            float64
2P             float64
2PA            float64
2P%            float64
eFG%           float64
FT             float64
FTA            float64
FT%            float64
ORB            float64
DRB            float64
TRB            float64
AST            float64
STL            float64
BLK            float64
TOV            float64
PF             float64
PTS            float64
Awards          object
Player Link     object
Season          object
PER            float64
TS%            float64
3PAr           float64
FTr            float64
ORB%           float64
DRB%           float64
TRB%           float64
AST%           float64
STL%           float64
BLK%           float64
TOV%           float64
USG%           float64
OWS        

Most of these columns consist of numbers already which is good. A few of the non-number columns are only for identification purposes, namely "Player", "identifier", and "Player Links". I can use the "Team" column as a feature, but implementing it would use a lot of columns, and I doubt it'd be very helful. I will however be turning "Season", "Awards", and "Pos" into features.

In [54]:
engined_val_sal = nonull_val_sal.copy()

In [55]:
finals_years = [float(i[0:2] + i[5:7]) for i in engined_val_sal["Season"]]
engined_val_sal["Finals Year"] = finals_years

In [56]:
engined_val_sal["PG Pos"] = (engined_val_sal["Pos"] == "PG").astype(int)
engined_val_sal["SG Pos"] = (engined_val_sal["Pos"] == "SG").astype(int)
engined_val_sal["SF Pos"] = (engined_val_sal["Pos"] == "SF").astype(int)
engined_val_sal["PF Pos"] = (engined_val_sal["Pos"] == "PF").astype(int)
engined_val_sal["C Pos"] = (engined_val_sal["Pos"] == "C").astype(int)

In [57]:
mod_awards = engined_val_sal["Awards"]
mod_awards[mod_awards.isnull()] = ''

In [58]:
awards_list = ["MVP", "DPOY", "ROY", "6MOY", "MIP", "CPOY", "AS", "NBA"]
for i in awards_list:
    if i == "AS":
        col_name = i
        reg = r'AS,|AS$' 
        matches = mod_awards.str.match(reg).astype(int)
        engined_val_sal[col_name] = matches
    elif i == "NBA":
        for j in np.arange(1, 4):
            col_name = i + str(j)
            reg = r'NBA' + str(j) + ',|NBA' + str(j) + '$' # e.g. for i = NBA and j = 1, reg = r'NBA1,|NBA1$'
            matches = mod_awards.str.match(reg).astype(int)
            engined_val_sal[col_name] = matches
    else:
        for j in np.arange(1, 6):
            col_name = i + '-' + str(j)
            reg = r'' + i + '-' + str(j) + ',|' + i + '-' + str(j) + '$' # e.g. for i = MVP and j = 2, reg = r'MVP-2,|MVP-2$'
            matches = mod_awards.str.match(reg).astype(int)
            engined_val_sal[col_name] = matches

In [59]:
engined_val_sal.columns

Index(['Player', 'Age', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Awards',
       'Player Link', 'Season', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Team', 'Lg', 'Salary',
       'Finals Year', 'PG Pos', 'SG Pos', 'SF Pos', 'PF Pos', 'C Pos', 'MVP-1',
       'MVP-2', 'MVP-3', 'MVP-4', 'MVP-5', 'DPOY-1', 'DPOY-2', 'DPOY-3',
       'DPOY-4', 'DPOY-5', 'ROY-1', 'ROY-2', 'ROY-3', 'ROY-4', 'ROY-5',
       '6MOY-1', '6MOY-2', '6MOY-3', '6MOY-4', '6MOY-5', 'MIP-1', 'MIP-2',
       'MIP-3', 'MIP-4', 'MIP-5', 'CPOY-1', 'CPOY-2', 'CPOY-3', 'CPOY-4',
       'CPOY-5', 'AS', 'NBA1', 'NBA2', 'NBA3'],
      dtype='object')

We can now drop our non-number columns and we will have a dataset ready to train with.

In [60]:
final_vals_sals = engined_val_sal.drop(["Player", "Season", "Team", "Pos", "Awards", "Player Link", "Lg"], axis = 1)

In [63]:
with open("stats_and_salaries.csv", "w") as file:
    file.write(final_vals_sals.to_csv())

## The Data We've Lost

Before we go ahead and train with our data we lost, as the data we lost (and kept) may be a source of bias. 